In [1]:
import pyinterp
import netCDF4
import numpy as np
import matplotlib.pylab as plt
import glob
from yaml import load, Loader
import scipy.signal
#import matplotlib.dates as dates
import sys
sys.path.append('/home/ad/ballarm/scratch/OSE_intercomparison/scuba/src')  

from scuba.src.mod_constant import *
from scuba.src.mod_segmentation import *
from scuba.src.mod_spectral import *

from src.mod_mission import *
from src.mod_write import *
from src.mod_filter import *
from src.mod_grid import *
from src.mod_alongtrack import *

In [2]:
# independent along-track
residus_files_cmems = '/home/ad/ballarm/scratch/data/CMEMS/dt-along-track/zarr/c2'
mission = 'c2'
mission_management = '/home/ad/ballarm/tools/scuba/share/MissionManagement.yaml'
# study area
lon_min = 0.
lon_max = 360.
lat_min = -90.
lat_max = 90.
is_circle = False
time_min = '2017-01-01'
time_max = '2017-12-31'
bin_lat_step = 1.
bin_lon_step = 1.
bin_time_step = '1D'

In [3]:
# DUACS maps
grid_files_duacs = '/home/ad/ballarm/scratch/OSE_intercomparison/data/OSE_GULFSTREAM_FPGENN_clean.nc'
var2add_duacs = ['OI'] #  
var2sub_duacs = []
YAML = load(open(str('scuba_alongtrack_DUACS.yaml')), Loader=Loader)
output_filename_duacs = 'psd_OSE_GULFSTREAM_DUACS_'+time_min+'-'+time_max+'_'+mission+'_vxxc.nc'

grid_files_miost = '/home/ad/ballarm/scratch/OSE_intercomparison/data/OSE_GULFSTREAM_MIOST.nc'
var2add_miost = ['Hls', 'Hss', 'Hb', 'MDH'] #  
var2sub_miost = []
YAML = load(open(str('scuba_alongtrack_DUACS.yaml')), Loader=Loader)
output_filename_miost = 'psd_OSE_GULFSTREAM_MIOST_'+time_min+'-'+time_max+'_'+mission+'_vxxc.nc'

grid_files_dymost_static = '/home/ad/ballarm/scratch/OSE_intercomparison/data/OSE_GULFSTREAM_DYMOST_STATIC.nc'
var2add_dymost = ['Ha']
var2sub_dymost = []
YAML = load(open(str('scuba_alongtrack_DUACS.yaml')), Loader=Loader)
output_filename_dymost_static = 'psd_OSE_GULFSTREAM_DYMOST_STATIC_'+time_min+'-'+time_max+'_'+mission+'_vxxc.nc'

grid_files_dymost_dynamic = '/home/ad/ballarm/scratch/OSE_intercomparison/data/OSE_GULFSTREAM_DYMOST_DYNAMIC.nc'
var2add_dymost = ['Ha']
var2sub_dymost = []
YAML = load(open(str('scuba_alongtrack_DUACS.yaml')), Loader=Loader)
output_filename_dymost_dyn = 'psd_OSE_GULFSTREAM_DYMOST_DYNAMIC_'+time_min+'-'+time_max+'_'+mission+'_vxxc.nc'

grid_files_dymost_dynamic_1_8 = '/home/ad/ballarm/scratch/OSE_intercomparison/data/OSE_GULFSTREAM_DYMOST_DYNAMIC_1_8.nc'
var2add_dymost = ['Ha']
var2sub_dymost = []
YAML = load(open(str('scuba_alongtrack_DUACS.yaml')), Loader=Loader)
output_filename_dymost_dyn_1_8 = 'psd_OSE_GULFSTREAM_DYMOST_DYNAMIC_1_8_'+time_min+'-'+time_max+'_'+mission+'_vxxc.nc'


grid_files_fpgenn = '/home/ad/ballarm/scratch/OSE_intercomparison/data/OSE_GULFSTREAM_FPGENN_clean.nc'
var2add_fpgenn = ['FP-GENN'] #  
var2sub_fpgenn = []
YAML = load(open(str('scuba_alongtrack_DUACS.yaml')), Loader=Loader)
output_filename_fpgenn = 'psd_OSE_GULFSTREAM_FP-GENN_'+time_min+'-'+time_max+'_'+mission+'_vxxc.nc'

grid_files_bfn = '/home/ad/ballarm/scratch/OSE_intercomparison/data/OSE_GULFSTREAM_BFN.nc'
var2add_bfn = ['SSH'] #  
var2sub_bfn = []
YAML = load(open(str('scuba_alongtrack_DUACS.yaml')), Loader=Loader)
output_filename_bfn = 'psd_OSE_GULFSTREAM_BFN_'+time_min+'-'+time_max+'_'+mission+'_vxxc.nc'

grid_files_bfn_v0 = '/home/ad/ballarm/scratch/OSE_intercomparison/data/OSE_GULFSTREAM_BFN_v0.nc'
var2add_bfn = ['SSH'] #  
var2sub_bfn = []
YAML = load(open(str('scuba_alongtrack_DUACS.yaml')), Loader=Loader)
output_filename_bfn_v0 = 'psd_OSE_GULFSTREAM_BFN_v0_'+time_min+'-'+time_max+'_'+mission+'_vxxc.nc'

In [4]:
flag_duacs = False
flag_miost = False
flag_dymost_static = False
flag_dymost_dynamic = False
flag_dymost_dynamic_1_8 = False
flag_fpgenn = False
flag_bfn = False
flag_bfn_v0 = True

In [5]:
# Read SSH grid DUACS
if flag_duacs:
    output_filename = output_filename_duacs 
    x_axis, y_axis, z_axis, grid = fpgenn_grid_dataset(grid_files_duacs, var2add_duacs, var2sub_duacs,
                                                       lon_min=lon_min, lon_max=lon_max, 
                                                       lat_min=lat_min, lat_max=lat_max, 
                                                       time_min=time_min, time_max=time_max,
                                                       is_circle=is_circle) 
    
    #x_axis, y_axis, z_axis, grid = duacs_grid_dataset(grid_files_duacs, 
    #                                                  lon_min=lon_min, lon_max=lon_max, 
    #                                                  lat_min=lat_min, lat_max=lat_max, 
    #                                                  time_min=time_min, time_max=time_max,
    #                                                  is_circle=is_circle)

In [6]:
# Read SSH grid MIOST
if flag_miost:
    output_filename = output_filename_miost
    x_axis, y_axis, z_axis, grid = miost_grid_dataset(grid_files_miost, var2add_miost, var2sub_miost,
                                                      lon_min=lon_min, lon_max=lon_max, 
                                                      lat_min=lat_min, lat_max=lat_max, 
                                                      time_min=time_min, time_max=time_max,
                                                      is_circle=is_circle)

In [7]:
# Read SSH grid DYMOST
if flag_dymost_static:
    output_filename = output_filename_dymost_static    
    x_axis, y_axis, z_axis, grid = dymost_grid_dataset(grid_files_dymost_static, var2add_dymost, var2sub_dymost,
                                                       lon_min=lon_min, lon_max=lon_max, 
                                                       lat_min=lat_min, lat_max=lat_max, 
                                                       time_min=time_min, time_max=time_max,
                                                       is_circle=is_circle)

In [8]:
# Read SSH grid DYMOST
if flag_dymost_dynamic:
    output_filename = output_filename_dymost_dyn    
    x_axis, y_axis, z_axis, grid = dymost_grid_dataset(grid_files_dymost_dynamic, var2add_dymost, var2sub_dymost,
                                                       lon_min=lon_min, lon_max=lon_max, 
                                                       lat_min=lat_min, lat_max=lat_max, 
                                                       time_min=time_min, time_max=time_max,
                                                       is_circle=is_circle)

In [9]:
# Read SSH grid DYMOST
if flag_dymost_dynamic_1_8:
    output_filename = output_filename_dymost_dyn_1_8    
    x_axis, y_axis, z_axis, grid = dymost_grid_dataset(grid_files_dymost_dynamic_1_8, var2add_dymost, var2sub_dymost,
                                                       lon_min=lon_min, lon_max=lon_max, 
                                                       lat_min=lat_min, lat_max=lat_max, 
                                                       time_min=time_min, time_max=time_max,
                                                       is_circle=is_circle)

In [10]:
if flag_fpgenn:
    output_filename = output_filename_fpgenn    
    x_axis, y_axis, z_axis, grid = fpgenn_grid_dataset(grid_files_fpgenn, var2add_fpgenn, var2sub_fpgenn,
                                                       lon_min=lon_min, lon_max=lon_max, 
                                                       lat_min=lat_min, lat_max=lat_max, 
                                                       time_min=time_min, time_max=time_max,
                                                       is_circle=is_circle)    

In [11]:
if flag_bfn:
    output_filename = output_filename_bfn    
    x_axis, y_axis, z_axis, grid = bfn_grid_dataset(grid_files_bfn, var2add_bfn, var2sub_bfn,
                                                       lon_min=lon_min, lon_max=lon_max, 
                                                       lat_min=lat_min, lat_max=lat_max, 
                                                       time_min=time_min, time_max=time_max,
                                                       is_circle=is_circle) 

In [13]:
if flag_bfn_v0:
    output_filename = output_filename_bfn_v0    
    x_axis, y_axis, z_axis, grid = bfn_grid_dataset(grid_files_bfn_v0, var2add_bfn, var2sub_bfn,
                                                       lon_min=lon_min, lon_max=lon_max, 
                                                       lat_min=lat_min, lat_max=lat_max, 
                                                       time_min=time_min, time_max=time_max,
                                                       is_circle=is_circle) 

In [14]:
# Read along-track
ds_alongtrack = cmems_alongtrack_dataset(residus_files_cmems, 
                                          lon_min=lon_min, lon_max=lon_max, 
                                          lat_min=lat_min, lat_max=lat_max, 
                                          time_min=time_min, time_max=time_max)

In [15]:
# Interpolate maps on alongtrack
map_interp = pyinterp.trivariate(grid, 
                                 ds_alongtrack["longitude"].values, 
                                 ds_alongtrack["latitude"].values,
                                 z_axis.safe_cast(ds_alongtrack.time.values),
                                 bounds_error=False).reshape(ds_alongtrack["longitude"].values.shape)

In [16]:
ssh_alongtrack = ds_alongtrack["SLA"].values + ds_alongtrack["mdt"].values
lon_alongtrack = ds_alongtrack["longitude"].values
lat_alongtrack = ds_alongtrack["latitude"].values
time_alongtrack = ds_alongtrack["time"].values

In [17]:
# get mask from map_interp
msk1 = np.ma.masked_invalid(ssh_alongtrack).mask
msk2 = np.ma.masked_invalid(map_interp).mask
msk = msk1+msk2

ssh_alongtrack = np.ma.masked_where(msk, ssh_alongtrack).compressed()
lon_alongtrack = np.ma.masked_where(msk, lon_alongtrack).compressed()
lat_alongtrack = np.ma.masked_where(msk, lat_alongtrack).compressed()
time_alongtrack = np.ma.masked_where(msk, time_alongtrack).compressed()
map_interp = np.ma.masked_where(msk, map_interp).compressed()

In [18]:
# make time vector as days since 1950-01-01
time_alongtrack = (time_alongtrack - np.datetime64('1950-01-01T00:00:00Z')) / np.timedelta64(1, 'D')

/softs/rh7/conda/4.5.4/envs/pangeo_full_201909/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  


In [19]:
ref_segment, lon_segment, lat_segment, delta_x, npt, study_segment = compute_segment_alongtrack(ssh_alongtrack,
                                                                                                lon_alongtrack,
                                                                                                lat_alongtrack,
                                                                                                time_alongtrack,
                                                                                                YAML,
                                                                                                map_interp)

In [20]:
# Power spectrum density reference field
global_wavenumber, global_psd_ref = scipy.signal.welch(np.asarray(ref_segment).flatten(),
                                                           fs=1.0 / delta_x,
                                                           nperseg=npt,
                                                           scaling='density',
                                                           noverlap=0)

    # Power spectrum density study field
_, global_psd_study = scipy.signal.welch(np.asarray(study_segment).flatten(),
                                             fs=1.0 / delta_x,
                                             nperseg=npt,
                                             scaling='density',
                                             noverlap=0)

    # Power spectrum density study field
_, global_psd_diff = scipy.signal.welch(np.asarray(study_segment).flatten()-np.asarray(ref_segment).flatten(),
                                             fs=1.0 / delta_x,
                                             nperseg=npt,
                                             scaling='density',
                                             noverlap=0)

In [21]:
ds = xr.Dataset({"psd_ref": (["wavenumber"], global_psd_ref),
                 "psd_study": (["wavenumber"], global_psd_study),
                 "psd_diff": (["wavenumber"], global_psd_diff),
                },
                coords={"wavenumber": (["wavenumber"], global_wavenumber),
                       },
               )
ds.to_netcdf(output_filename)